In [ ]:
import numpy as np
import h5py as h5
from pyproj import Proj,transform
from osgeo import gdal
from osgeo import osr
from math import floor
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Import functions from pilot_functions.py


In [ ]:
# Read GEDI data
gedi = readGEDI('fileList')

# Reproject GEDI data to Sentinel2 epsg
gedi.reprojectGEDI(sent[0].epsg)

In [ ]:
#  Identify Sentinel2 pixels that intersect with GEDI footprints
xInd = np.array(np.floor((gedi.x - sent[0].xOrigin) / sent[0].pixelWidth), dtype = int)
yInd = np.array(np.floor((gedi.y - sent[0].yOrigin) / sent[0].pixelHeight), dtype = int)


# Prepare arrays to hold intersected Sentinel2 data
nGedi = gedi.cover.shape[0]
nSent = sent.shape[0]
gedi.s2out = np.full((nGedi, nSent), -999, dtype = float)


# Loop over GEDI footprints
for j in range(0, gedi.cover.shape[0]):
    if((xInd[j] >= 0) & (xInd[j] < sent[0].nX) & (yInd[j] >= 0) & (yInd[j] < sent[0].nY)):
        # Loop over Sentinel2 bands
        i = 0
        for i in range(0,nSent):
            gedi.s2out[j,i] = sent[i].data[xInd[j],yInd[j]]
            print(i, j, sent[i].data[xInd[j], yInd[j]])
            i = i + 1
                       
# Write csv 
# GEDI (reprojected x and y, sensitivity, cover) + Sentinel2 (coarsened, intersecting pixels for 9 bands)   
file = 'pilot.csv'
gedi.dumpData(file)

In [ ]:
# Calculate Sentinel2 NDVI
print(sent)

band4 = sent[2].data
band8 = sent[6].data

print(band4, band8)
print(gedi.s2out[0].data, gedi.s2out[1].data)

ndvi = np.full(gedi.s2out.shape[0], -999, dtype = float)
mask = [(band4 > 0) & (band8 > 0) & (gedi.s2out[0].data > 0) & (gedi.s2out[1].data > 0)]
ndvi = ((band8[mask] - band4[mask]) / (band8[mask] + band4[mask]))
print(ndvi)



#plt.imshow(ndvi, cmap = 'Greens')
#plt.colorbar()
#plt.show()


# Plot NDVI against GEDI cover

